In [ ]:
IRuby::Kernel.instance.switch_backend!(:pry)

require 'active_record'
require "activerecord-sqlserver-adapter"
require "awesome_print"

In [ ]:
ActiveRecord::Base.logger = ActiveSupport::Logger.new(STDOUT)
#ActiveRecord::Base.logger = ActiveSupport::Logger.new('log.txt')
#ActiveSupport::LogSubscriber.colorize_logging = false

In [ ]:
ActiveRecord::Base.establish_connection(
    adapter: 'sqlserver',
    encoding: 'utf8',
    host: 'test-host.com',
    database: 'TestDB'
)

In [ ]:
class DBRecord < ActiveRecord::Base
    self.abstract_class = true

    def self.table_name
      "app.#{name.demodulize}"
    end

end

In [ ]:
class DB
    class DBTable < DBRecord
    end
end

In [ ]:
DB::DBTable.columns.each { |column|
  puts column.name.to_s + ":" + column.type.to_s
}

In [ ]:
ap TestClass::DBTable.last(2)

In [ ]:
def pp(query)
    IRuby.display IRuby.table(query)
end

pp Ed::DBTable.last(2).as_json

In [ ]:
def cs(clss,str)
    ap clss.columns.map(&:name).select { |e| e.downcase.include?(str) }
end

cs BusinessUnit, "id"

In [ ]:
#find_by
DB::DBTable.find_by(:name => "Test BU", active: true).name

In [ ]:
#select
DB::DBTable.select(:name).where(name: "Test BU", active: true)
#Relation

Contract
.select(
:code, 
'customer.name'
)
.joins(:customer)
.joins(:contractadditionalinfo)
.left_outer_joins(:revenuerecognitiontype
)
.where(ContractID: "TestID")

In [ ]:
#pluck
Ed::DBTable.where(name: "Test BU", active: true).pluck(:id, :name)
#Array of arrays

In [ ]:
#find_by_sql
DB::DBTable.find_by_sql("select * from app.DBTable where name = 'Test BU'")
#Array

In [ ]:
#like, find_each
DB::DBTable.where("name like '%Test BU%'").find_each { |bu| puts bu.name; puts bu.class }

In [ ]:
#explain, distinct
DB::DBTable.where(name: "Test BU", active: true).distinct.explain
#DB::DBTable.where(name: "Test BU", active: true).to_sql

In [ ]:
#IN

Customer.where(orders_count: [1, 3, 5])

# not
Customer.where.not(orders_count: [1, 3, 5])

# OR
Customer.where(last_name: 'Smith').or(Customer.where(orders_count: [1, 3, 5]))

In [ ]:
#exists

Customer.exists?(first_name: ['Jane', 'Sergei'])

Customer.where(first_name: 'Ryan').exists?

In [ ]:
# Dates

ap DB::DBTable.find_by("createDate > '2024-03-01'") #returns only first record
DB::DBTable.where('createDate > ?', '2024-03-01').size #4

Book.where(created_at: (Time.now.midnight - 1.day)..Time.now.midnight)

Book.where(created_at: (Time.now.midnight - 1.day)..) #endless range
#SELECT * FROM books WHERE books.created_at >= '2008-12-21 00:00:00'

In [ ]:
#order
ap DB::DBTable.select(:name).where("createDate > '2024-03-01'").order(name: :asc).as_json

In [ ]:
# group
Order.group(:status).count
#{"being_packed"=>7, "shipped"=>12}

# having
big_orders = Order.select("created_at, sum(total) as total_price")
                  .group("created_at")
                  .having("sum(total) > ?", 200)

big_orders[0].total_price
# Returns the total price for the first Order object

In [ ]:
require 'csv'

data = DB::DBTable.last(3).as_json

# Define the headers based on the keys of the first hash
headers = data.first.keys

# Write data to CSV file with tab as separator
CSV.open('output.csv', 'w', col_sep: "\t", write_headers: true, headers: headers) do |csv|
  data.each do |row|
    csv << row.values
  end
end

puts "CSV file generated successfully."

In [ ]:
#update with iteration

DB::DBTable.where(name: "Test BU", active: true).each { 
    |bu| bu.name = "Test BU2" 
    #bu.save
}

#update_all bypasses validation

In [ ]:
# readonly

customer = Customer.readonly.first
customer.visits += 1
customer.save # Raises an ActiveRecord::ReadOnlyRecord

In [ ]:
# Relationships

belongs_to #FK is in current table

has_one
has_one :through

has_many #FK is in foreign table - calls back to current table
has_many :through

has_and_belongs_to_many

#

class Author < ApplicationRecord
    has_many :confirmed_books, -> { where "confirmed = 1" },
      class_name: "Book"
end

#

class Like < ActiveRecord::Base
    #belongs_to does not have to be defined, if we do not go to user/product from Like
    belongs_to :user 
    belongs_to :product
end

class User < ActiveRecord::Base
    has_many :likes
    has_many :products, through: :likes
end

#same
class User < ActiveRecord::Base
    has_many :likes
    has_many :liked_products, through: :likes, source: :product
end

In [ ]:
# Joins

author = Author.first
Book.where(author: author)
Author.joins(:books).where(books: { author: author })

Author.joins("INNER JOIN books ON books.author_id = authors.id AND books.out_of_print = FALSE")
"SELECT authors.* FROM authors INNER JOIN books ON books.author_id = authors.id AND books.out_of_print = FALSE"

Book.joins(:reviews)
"SELECT books.* FROM books
INNER JOIN reviews ON reviews.book_id = books.id"

Book.joins(:author, :reviews)
"SELECT books.* FROM books
INNER JOIN authors ON authors.id = books.author_id
INNER JOIN reviews ON reviews.book_id = books.id"

Book.joins(reviews: :customer) #nested
"SELECT books.* FROM books
INNER JOIN reviews ON reviews.book_id = books.id
INNER JOIN customers ON customers.id = reviews.customer_id"

Author.joins(books: [{ reviews: { customer: :orders } }, :supplier]) #nested
"SELECT authors.* FROM authors
INNER JOIN books ON books.author_id = authors.id
INNER JOIN reviews ON reviews.book_id = books.id
INNER JOIN customers ON customers.id = reviews.customer_id
INNER JOIN orders ON orders.customer_id = customers.id
INNER JOIN suppliers ON suppliers.id = books.supplier_id"

Customer.left_outer_joins(:reviews).distinct.select('customers.*, COUNT(reviews.*) AS reviews_count').group('customers.id')
"SELECT DISTINCT customers.*, COUNT(reviews.*) AS reviews_count FROM customers
LEFT OUTER JOIN reviews ON reviews.customer_id = customers.id GROUP BY customers.id"

#Examples
time_range = (Time.now.midnight - 1.day)..Time.now.midnight
Customer.joins(:orders).where('orders.created_at' => time_range).distinct

In [ ]:
# where.associated and where.missing

Customer.where.associated(:reviews)
"SELECT customers.* FROM customers
INNER JOIN reviews ON reviews.customer_id = customers.id
WHERE reviews.id IS NOT NULL"

Customer.where.missing(:reviews)
"SELECT customers.* FROM customers
LEFT OUTER JOIN reviews ON reviews.customer_id = customers.id
WHERE reviews.id IS NULL"

In [ ]:
# includes and eager_load

books = Book.limit(10)
books.each do |book|
  puts book.author.last_name
end

#The above code executes 1 (to find 10 books) + 10 (one per each book to load the author) = 11 queries in total.

books = Book.includes(:author).limit(10)
books.each do |book|
  puts book.author.last_name
end

"The above code will execute just 2 queries, as opposed to the 11 queries from the original case:
SELECT books.* FROM books LIMIT 10
SELECT authors.* FROM authors
WHERE authors.id IN (1,2,3,4,5,6,7,8,9,10)"

books = Book.eager_load(:author).limit(10) #uses left join
books.each do |book|
  puts book.author.last_name
end

"SELECT DISTINCT books.id FROM books LEFT OUTER JOIN authors ON authors.id = books.author_id LIMIT 10
SELECT books.id AS t0_r0, books.last_name AS t0_r1, ...
FROM books LEFT OUTER JOIN authors ON authors.id = books.author_id
WHERE books.id IN (1,2,3,4,5,6,7,8,9,10)"

In [ ]:
# Scopes

class Book < ApplicationRecord
    scope :out_of_print, -> { where(out_of_print: true) }
end
Book.out_of_print

#chaining
class Book < ApplicationRecord
    scope :out_of_print, -> { where(out_of_print: true) }
    scope :out_of_print_and_expensive, -> { out_of_print.where("price > 500") }
end

#arguments
class Book < ApplicationRecord
    scope :costs_more_than, ->(amount) { where("price > ?", amount) }
end
Book.costs_more_than(100.10)

#same as:
class Book < ApplicationRecord
    def self.costs_more_than(amount)
      where("price > ?", amount)
    end
end

"However, there is one important caveat: A scope will always return an ActiveRecord::Relation object, 
even if the conditional evaluates to false, whereas a class method, will return nil. 
This can cause NoMethodError when chaining class methods with conditionals, if any of the conditionals return false"

#conditions
class Order < ApplicationRecord
    scope :created_before, ->(time) { where(created_at: ...time) if time.present? }
end

#Default Scope
class Book < ApplicationRecord
    default_scope { where(out_of_print: false) }
end

In [ ]:
# enum

class Order < ApplicationRecord
    enum :status, [:shipped, :being_packaged, :complete, :cancelled]
end

Order.shipped
order.shipped?
order.shipped!

Order.shipped.any?

Order.shipped.many?
# SELECT COUNT(*) FROM (SELECT 1 FROM orders WHERE orders.status = 0 LIMIT 2)

In [ ]:
# Transactions

# Perform a transaction and either commit or rollback depending on
# whether an exception was raised.
ActiveRecord::Base.transaction do
    # Do stuff here like updating, creating, or deleting records.
  
    # If there are any exceptions raised, the database commit will not happen,
    # and it will perform a rollback restoring the database.
  
    # Nested transactions are also possible.
  
    # However, the actual Ruby object will be in a dirty state and not restored
    # to the original state before the attempted transaction was started
  end

User.transaction do
    users_to_update = User.where('age > ?', 18)
    users_to_update.each do |user|
      user.status = 'active'
      user.save
    end
end

#update with rollback and count affected rows
ActiveRecord::Base.transaction do
  num_rows_affected = DBTable.where(name: "Test BU").update(Status: 16)
  p num_rows_affected.size
  raise ActiveRecord::Rollback
end

In [ ]:
# Examples

Customer
  .select('customers.id, customers.last_name, reviews.body')
  .joins(:reviews)
  .where('reviews.created_at > ?', 1.week.ago)

"SELECT customers.id, customers.last_name, reviews.body
FROM customers
INNER JOIN reviews ON reviews.customer_id = customers.id
WHERE (reviews.created_at > '2019-01-08')"

#

Book
  .select('books.id, books.title, authors.first_name')
  .joins(:author)
  .find_by(title: 'Abstraction and Specification in Program Development')

"SELECT books.id, books.title, authors.first_name
FROM books
INNER JOIN authors ON authors.id = books.author_id
WHERE books.title = $1 [["title", "Abstraction and Specification in Program Development"]]
LIMIT 1"

#

class Customer < ApplicationRecord
  def name
    "I am #{first_name}"
  end
end

Customer.select(:first_name).map &:name
#["I am David", "I am Jeremy", "I am Jose"]

#

Order.joins(:customer, :books).pluck("orders.created_at, customers.email, books.title")